## Import Python libary

In [27]:
import librosa
import os
import re
import numpy as np
import matplotlib.pyplot as plt

## Global variable

In [28]:
AudioFiles = []  # 包含所有的音檔

## Read file

In [29]:
def ReadFile(AudioFiles):
    base_path = 'dataset'
    for actor_id in os.listdir(base_path):
        actor_path = os.path.join(base_path, actor_id)     # actor_path = dataset/id
        tmp = []
        
        for book_id in os.listdir(actor_path):
            book_path = os.path.join(actor_path, book_id)  # book_path = dataset/id/number
            
            for file_name in os.listdir(book_path):
                if file_name.endswith('.flac'):
                    file_path = os.path.join(book_path, file_name)  # file_path = dataset/id/number/file_name

                    # 判斷音檔長度，大於10秒才保留
                    duration = librosa.get_duration(path=file_path)
                    if duration < 10: continue
                    
                    # 使用librosa加载音频
                    y, sr = librosa.load(file_path, sr=None)
                    tmp.append((y, sr))
                    
        AudioFiles.append(tmp)

ReadFile(AudioFiles)


## VAD and Noise reduction

In [ ]:
def VAD():
    pass

def NoiseReduction():
    pass

In [ ]:
# The path of the dataset folder
base_path = 'dataset'

for actor_id in os.listdir(base_path):
    actor_path = os.path.join(base_path, actor_id)     #actor_path = dataset/19
    
    for book_id in os.listdir(actor_path):
        book_path = os.path.join(actor_path, book_id)  #actor_path = dataset/19/198
        
        melspectrograms = []
        count = 0
        
        for file_name in os.listdir(book_path):
            if file_name.endswith('.flac'):
                file_path = os.path.join(book_path, file_name)

                # 判斷音檔長度，大於10秒才保留
                duration = librosa.get_duration(filename=file_path)
                if duration < 10: continue
                else: count += 1
                
                # 使用librosa加载音频
                y, sr = librosa.load(file_path, sr=None)
                
                # 計算梅爾頻譜圖
                melspectrogram = librosa.feature.melspectrogram(y=y, sr=sr)
                # print(melspectrogram)
                # print(melspectrogram.shape)

                # 將梅爾頻譜圖轉換為分貝表示
                log_melspectrogram = librosa.power_to_db(melspectrogram, ref=np.max)

                # 顯示梅爾頻譜圖
                librosa.display.specshow(log_melspectrogram, sr=sr, x_axis='time', y_axis='mel')
                plt.colorbar(format='%+2.0f dB')
                plt.show()

                # 提取MFCC参数
                # mfccs = librosa.feature.mfcc(y=y, sr=sr)
                print(f"Actor: {actor_id}, Book: {book_id}, File: {file_name}, MFCC Shape: {melspectrogram.shape}")
                # plt.figure(figsize=(10, 4))
                # librosa.display.specshow(mfccs, x_axis='time', y_axis='mel')
                # plt.colorbar()
                # plt.title(f"MFCC: {file_name}")
                # plt.show()
                
                # all_mfccs.append(mfccs)
                # melspectrograms.append(melspectrogram)

        print(f"count: {count}")
        
        # # 将所有的MFCC参数合并为一个numpy数组
        # all_mfccs = np.concatenate(all_mfccs, axis=1)
        
        # # 保存MFCC参数为.npy文件
        # mfccs_path = f'{actor_path}/mfccs'
        # if not os.path.exists(mfccs_path):
        #     os.makedirs(mfccs_path)

        # file_id = file_name.split('.')[0]
        # np.save(os.path.join(mfccs_path, f'{file_id}.npy'), all_mfccs)


Words counts

In [ ]:
target_words = ["ZERO", "ONE", "TWO", "THREE", "FOUR", "FIVE", "SIX", "EIGHT", "NINE", "TEN"]
dataset_root = 'testest-code'
word_counts = {word: 0 for word in target_words}

actorNums = 0
chapterNums = 0
for actor_folder in os.listdir(dataset_root):
    actor_path = os.path.join(dataset_root, actor_folder)
    
    if os.path.isdir(actor_path):

        actorNums += 1
        # print(f'Actor Folder: {actor_folder}')
        
        for script_folder in os.listdir(actor_path):
            script_path = os.path.join(actor_path, script_folder)
            
            if os.path.isdir(script_path):
                trans_file = os.path.join(script_path, f'{actor_folder}-{script_folder}.trans.txt')
                
                if os.path.exists(trans_file):
                    chapterNums += 1
                    with open(trans_file, 'r', encoding='utf-8') as f:
                        script_text = f.read()
                    
                    for word in target_words:
                        word_counts[word] += len(re.findall(r'\b' + word + r'\b', script_text, re.IGNORECASE))


print(str(dataset_root))
print('Total actors   : ' + str(actorNums))
print('Total chapters : ' + str(chapterNums))
for word, count in word_counts.items():
    print(f'{word:{6}}: {count}')